In [1]:
#import sys
#!conda install --yes --prefix {sys.prefix} torchvision

In [2]:
#import sys
#!conda install --yes --prefix {sys.prefix} -c rdkit rdkit

In [1]:
from tensorboard_logger import configure, log_value
import tensorboard
import argparse
import os
import numpy as np
import math
import sys
import glob
import time
#import utils import Logger


import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch

### Part to use my own images

In [2]:
import glob
import random
import os
import numpy as np

from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

class ImageDataset(Dataset):
    def __init__(self, folder_path, transforms_=None):
        self.transform = transforms.Compose(transforms_)
        self.files = sorted(glob.glob('%s/*.*' % folder_path))

    def __getitem__(self, index):
        img = Image.open(self.files[index % len(self.files)])
        img = self.transform(img)
        return img

    def __len__(self):
        return len(self.files)

In [3]:
img_shape = (3, 100, 100)

cuda = True if torch.cuda.is_available() else False
print(cuda)

True


## Option Settings

In [4]:
n_epochs=5000 #number of epochs of training
batch_size=400 #size of the batches
lr=0.0002 #adam: learning rate
b1=0.5  #"adam: decay of first order momentum of gradient")
b2=0.999 #adam: decay of first order momentum of gradient")
n_cpu=8
latent_dim=100
img_size=200
channels=3
n_critic=5
clip_value=0.01
sample_interval=1000
img_shape = (channels, img_size, img_size)
crop_size = 400
print(img_shape)

(3, 200, 200)


### nn.modules

In [5]:

folder_path = "./zinc100k500px"
transforms_ = [ transforms.Resize(img_size),
                transforms.CenterCrop(img_size),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
dataloader = DataLoader(ImageDataset(folder_path, transforms_=transforms_),
                        batch_size=batch_size, shuffle=True, num_workers=8)

In [6]:
#img_shape = (3, 100, 100)

#folder_path = "./molecules100"
#transforms_ = [ transforms.Resize(100),
#                transforms.CenterCrop(100),
#                transforms.ToTensor(),
#                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
#dataloader = DataLoader(ImageDataset(folder_path, transforms_=transforms_),
#                        batch_size=100, shuffle=True, num_workers=8)

In [7]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(100, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img

In [8]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

In [9]:
# Loss weight for gradient penalty
lambda_gp = 10

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()


In [10]:
if cuda:
    generator.cuda()
    discriminator.cuda()

TO DO: Determine optimal learning rate

# Actual training now

In [11]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b2, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor


def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty



In [12]:
imgs=next(iter(dataloader))

In [13]:
# Sample noise as generator input

imgs
z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))
print(z)
print(latent_dim)
print(imgs.shape)
print(imgs)

# Generate a batch of images
fake_imgs = generator(z)


tensor([[-0.1455, -1.3897,  0.5656,  ..., -0.1336, -0.3261, -0.2717],
        [ 0.3769, -1.7770, -1.7421,  ...,  0.8083,  1.5583,  0.1583],
        [ 0.3128,  0.6326, -0.6259,  ...,  0.2174,  1.5046,  0.7104],
        ...,
        [-0.7455,  0.2533,  0.4738,  ...,  2.8316, -1.6945,  0.2717],
        [ 0.3215,  1.1270,  1.6873,  ...,  1.4072, -0.1512,  0.7300],
        [-0.5515,  1.2271,  1.1987,  ..., -0.7063,  0.1423, -1.6368]],
       device='cuda:0')
100
torch.Size([400, 3, 200, 200])
tensor([[[[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          ...,
          [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000]],

         [[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.00

In [14]:
batches_done

NameError: name 'batches_done' is not defined

In [23]:
# old logger
from utils import Logger
#configure("runs/test2",flush_secs=5)
#logger = Logger(model_name='wgangp1t1', data_name='200px')
#num_batches = len(dataloader)
n_epochs=1
batches_done = 0
### new logger


#batch_size=100
#print(latent_dim)
for epoch in range(n_epochs):
    for i, imgs in enumerate(dataloader):
        ## monitor time

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))
        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()
        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))
        # Generate a batch of images
        fake_imgs = generator(z)
        # Real images
        real_validity = discriminator(real_imgs)
        # Fake images
        fake_validity = discriminator(fake_imgs)
        # Gradient penalty
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
        # Adversarial loss
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty
        d_loss.backward()
        optimizer_D.step()
        optimizer_G.zero_grad()
        # Train the generator every n_critic steps
        if i % n_critic == 0:
            # -----------------
            #  Train Generator
            # -----------------
            # Generate a batch of images
            fake_imgs = generator(z)
            # Loss measures generator's ability to fool the discriminator
            # Train on fake images
            fake_validity = discriminator(fake_imgs)
            g_loss = -torch.mean(fake_validity)
            g_loss.backward()
            optimizer_G.step()
            


            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f] [5batch time: %f]"
                % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item(), batchtime)
            )

            #logger.log(d_loss, g_loss, epoch, batches_done, num_batches)
            if batches_done % sample_interval/10 ==0:
                log_value('g_loss', g_loss, batches_done)
                log_value('d_loss', d_loss, batches_done)
            
            if batches_done % sample_interval == 0:
                save_image(fake_imgs.data[:25], "molpics500px/%d_c.png" % batches_done, nrow=5, normalize=True)

            end = time.time()

            batchtime= end - start
            start = time.time()
            batches_done += n_critic

[Epoch 0/1] [Batch 0/250] [D loss: 36.971878] [G loss: -1377.578979] [5batch time: -0.000001]
[Epoch 0/1] [Batch 5/250] [D loss: 25.295403] [G loss: -1309.954346] [5batch time: 11.239845]
[Epoch 0/1] [Batch 10/250] [D loss: 16.670288] [G loss: -1236.936890] [5batch time: 1.651815]
[Epoch 0/1] [Batch 15/250] [D loss: 4.637108] [G loss: -1153.895386] [5batch time: 1.628217]
[Epoch 0/1] [Batch 20/250] [D loss: -3.039436] [G loss: -1068.142212] [5batch time: 1.603856]
[Epoch 0/1] [Batch 25/250] [D loss: -2.178219] [G loss: -985.437866] [5batch time: 1.603096]
[Epoch 0/1] [Batch 30/250] [D loss: -5.159782] [G loss: -894.653503] [5batch time: 1.565035]
[Epoch 0/1] [Batch 35/250] [D loss: -10.541723] [G loss: -798.574646] [5batch time: 1.568519]
[Epoch 0/1] [Batch 40/250] [D loss: -11.379900] [G loss: -703.054504] [5batch time: 1.572233]
[Epoch 0/1] [Batch 45/250] [D loss: -15.985483] [G loss: -602.522034] [5batch time: 1.580967]
[Epoch 0/1] [Batch 50/250] [D loss: -17.476208] [G loss: -504.7

Process Process-64:
Process Process-63:
Process Process-60:
Process Process-61:
Process Process-59:
Process Process-62:
Process Process-57:
Process Process-58:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/multi

  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/site-packages/torchvision/transforms/transforms.py", line 49, in __call__
    img = t(img)
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/site-packages/torchvision/transforms/transforms.py", line 175, in __call__
    return F.resize(img, self.size, self.interpolation)
  File "<ipython-input-2-3e5f53181d14>", line 17, in __getitem__
    img = self.transform(img)
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/site-packages/torchvision/transforms/functional.py", line 204, in resize
    return img.resize((ow, oh), interpolation)
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/site-packages/torchvision/transforms/transforms.py", line 49, in __call__
    img = t(img)
  File "/home/jgmeyer2/anaconda3/envs/vangan/lib/python3.6/site-packages/torchv

KeyboardInterrupt: 

KeyboardInterrupt


In [ ]:

#tensorboard
!python -m tensorboard.main --logdir runs
#tensorboard --help
#tensorboard --logdir=runs

#logger.d_loss.data.cpu().numpy()
#logger.display_status(epoch, n_epochs, batches_done, num_batches, d_loss, g_loss, d_pred_real, d_pred_fake)

### Save the models

In [69]:
os.makedirs("/home/jgmeyer2/vangan/gans/models",exist_ok=True)
PATH = "/home/jgmeyer2/vangan/gans/models/"
modelid="200px_668epoch"


state_g = {
    'epoch': epoch,
    'state_dict': generator.state_dict(),
    'optimizer': optimizer_G.state_dict()
    }
torch.save(state_g, PATH+"g"+modelid+".model")

state_d = {
    'epoch': epoch,
    'state_dict': discriminator.state_dict(),
    'optimizer': optimizer_D.state_dict()
    }
torch.save(state_d, PATH+"d"+modelid+".model")
print("saved models @")
print(epoch)


#def save_model(net, optim, ckpt_fname):
#    state_dict = net.module.state_dict()
#    for key in state_dict.keys():
#        state_dict[key] = state_dict[key].cpu()
#        torch.save({
#            'epoch': epoch,                                                                                                                                                                                     
#            'state_dict': state_dict,                                                                                                                                                                                
#            'optimizer': optim},                                                                                                                                                                                     
#            ckpt_fname)

saved models @
669


# Load models

In [70]:
state_g = torch.load("/home/jgmeyer2/vangan/gans/models/g5k.modelg5k.model")
state_d = torch.load("/home/jgmeyer2/vangan/gans/models/g5k.modeld5k.model")


generator.load_state_dict(state_g['state_dict'])
optimizer_G.load_state_dict(state_g['optimizer'])

discriminator.load_state_dict(state_d['state_dict'])
optimizer_D.load_state_dict(state_d['optimizer'])





RuntimeError: Error(s) in loading state_dict for Generator:
	size mismatch for model.11.weight: copying a param of torch.Size([120000, 1024]) from checkpoint, where the shape is torch.Size([30000, 1024]) in current model.
	size mismatch for model.11.bias: copying a param of torch.Size([120000]) from checkpoint, where the shape is torch.Size([30000]) in current model.